In [11]:
import pandas as pd
import arcpy
from arcpy.sa import *
import numpy as np
import arcgis
import requests
import os

Define variables for data paths:

In [12]:
# NDAWN url path for Average Bare Soil Temperature CSV
ndawn_url = r"https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=185&station=29&station=30&station=154&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=125&station=176&station=177&station=8&station=180&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110&variable=ddbst&year=2023&ttype=daily&quick_pick=&begin_date=2023-03-01&end_date=2023-03-01"

# Daily MN Weather Observations for February 
weather_url = r"https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&month=2&year=2023"

# MN NLCD Land Cover TIFF (MN Geo Commons)
landcover_path = r"C:\Users\cason\Documents\GIS5572\Lab2\NLCD_2019_Land_Cover.tif"

# MN 30m Digital Elevation Model (MN Geo Commons)
dem = r"C:\Users\cason\Documents\GIS5572\Lab2\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m"

# Local GDB path for local output storage and for mirroring to PostgreSQL database
local_gdb = r"C:\Users\cason\Documents\GIS5572\Lab2\Lab2.gdb"

# NDAWN Data

Load CSV from url:

In [13]:
# Load CSV from NDAWN url and clean up unneeded rows
df = pd.read_csv(ndawn_url, skiprows=[0, 1, 2, 4])

# Display first five rows of table
df.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Bare Soil Temp,Avg Bare Soil Temp Flag
0,Ada,47.321190,-96.514060,910,2023,3,1,30.274,NaN
1,Adams,48.499880,-98.075880,1580,2023,3,1,22.644,NaN
2,Alamo,48.546520,-103.471860,2157,2023,3,1,29.130,NaN
3,Alexander,47.750560,-103.733580,2202,2023,3,1,31.271,NaN
4,Alvarado,48.245942,-97.021532,809,2023,3,1,29.482,NaN


QA/QC:

In [14]:
# Define the latitude and longitude ranges for Minnesota
min_lat = 43.5
max_lat = 49.5
min_lon = -97.5
max_lon = -89.5

# Filter the dataframe to only include locations in Minnesota
mn_df = df[(df['Latitude'] >= min_lat) & (df['Latitude'] <= max_lat) & (df['Longitude'] >= min_lon) & (df['Longitude'] <= max_lon)]

# Display first five rows of updated table
mn_df.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Bare Soil Temp,Avg Bare Soil Temp Flag
0,Ada,47.321190,-96.514060,910,2023,3,1,30.274,NaN
4,Alvarado,48.245942,-97.021532,809,2023,3,1,29.482,NaN
8,Ayr,47.046390,-97.494810,1215,2023,3,1,31.171,NaN
13,Becker,45.343990,-93.850140,942,2023,3,1,28.973,NaN
23,Campbell,46.064941,-96.370141,987,2023,3,1,30.010,NaN


In [15]:
# Drop rows that contain outliers over 1 standard deviation above the mean
numMean = mn_df["Avg Bare Soil Temp"].mean()
numStd = mn_df["Avg Bare Soil Temp"].std()

mn_df = mn_df.loc[mn_df["Avg Bare Soil Temp"].between(numMean - numStd, numMean + numStd)]

# Display table with updated rows
mn_df

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Bare Soil Temp,Avg Bare Soil Temp Flag
0,Ada,47.321190,-96.514060,910,2023,3,1,30.274,NaN
4,Alvarado,48.245942,-97.021532,809,2023,3,1,29.482,NaN
8,Ayr,47.046390,-97.494810,1215,2023,3,1,31.171,NaN
13,Becker,45.343990,-93.850140,942,2023,3,1,28.973,NaN
23,Campbell,46.064941,-96.370141,987,2023,3,1,30.010,NaN
29,Clarissa,46.111545,-94.905826,1304,2023,3,1,26.301,NaN
48,Ekre,46.540050,-97.140940,1053,2023,3,1,30.783,NaN
51,Emerado,47.912010,-97.325150,877,2023,3,1,26.882,NaN
54,Fargo,46.896932,-96.812209,902,2023,3,1,27.932,NaN
62,Fox,48.877750,-95.850170,1040,2023,3,1,29.321,NaN


CSV to Feature Class Conversion:

In [16]:
# Convert from DF to SEDF
ndawn_sedf = arcgis.GeoAccessor.from_xy(mn_df, "Longitude", "Latitude")

In [17]:
# Convert from SEDF to Feature Class
ndawn_sedf.spatial.to_featureclass(location=os.path.join(local_gdb, "bare_soil_observations"))

'C:\\Users\\cason\\Documents\\GIS5572\\Lab2\\Lab2.gdb\\bare_soil_observations'

# Weather Data

Load data from url:

In [18]:
weather_response = requests.get(weather_url)
weather_response

<Response [200]>

Display as JSON:

In [19]:
weather_json = weather_response.json()["features"]
weather_json

[{'type': 'Feature', 'properties': {'station': 'MN001', 'date': '2023-02-01', 'max_tmpf': 20.119978, 'min_tmpf': -0.40001097, 'precip': None, 'max_gust': 14.580777, 'snow': None, 'snowd': None, 'min_rh': 57.0, 'max_rh': 84.0, 'max_dwpf': 8.960011, 'min_dwpf': -5.0620046, 'min_feel': -12.071555, 'avg_feel': None, 'max_feel': 15.251119, 'max_drct': None, 'precip_est': False, 'tmpf_est': False, 'max_gust_localts': '2023-02-01T01:40:00Z', 'temp_hour': None, 'avg_sknt': None, 'vector_avg_drct': None, 'min_rstage': None, 'max_rstage': None, 'id': 'MN001', 'name': 'Twin Lakes I-35 Mile Post 1'}, 'geometry': {'type': 'Point', 'coordinates': [-93.354057312, 43.5083312988]}}, {'type': 'Feature', 'properties': {'station': 'MN002', 'date': '2023-02-01', 'max_tmpf': 13.460011, 'min_tmpf': -9.039989, 'precip': None, 'max_gust': 12.440605, 'snow': None, 'snowd': None, 'min_rh': 60.0, 'max_rh': 86.0, 'max_dwpf': 4.5680065, 'min_dwpf': -13.25201, 'min_feel': -9.039989, 'avg_feel': None, 'max_feel': 13.

Convert to data frame and display table:

In [20]:
weather_raw = pd.DataFrame.from_records(weather_json)
weather_raw

,type,properties,geometry
0,Feature,"{'station': 'MN001', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-93.35405731..."
1,Feature,"{'station': 'MN002', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-94.1191, 44..."
2,Feature,"{'station': 'MN003', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-93.29242706..."
3,Feature,"{'station': 'MN004', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-92.99275207..."
4,Feature,"{'station': 'MN005', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-92.83856201..."
...,...,...,...
4279,Feature,"{'station': 'MN158', 'date': '2023-02-28', 'ma...","{'type': 'Point', 'coordinates': [-96.27693176..."
4280,Feature,"{'station': 'MN159', 'date': '2023-02-28', 'ma...","{'type': 'Point', 'coordinates': [-94.81175994..."
4281,Feature,"{'station': 'MN160', 'date': '2023-02-28', 'ma...","{'type': 'Point', 'coordinates': [-96.04634094..."
4282,Feature,"{'station': 'MN161', 'date': '2023-02-28', 'ma...","{'type': 'Point', 'coordinates': [-93.76606750..."


Define function that extracts desired fields from properties:

In [21]:
def extractFields(field):
    weather_raw[field] = weather_raw["properties"].apply(lambda x: dict(x)[field])

List of desired fields:

In [22]:
weather_properties = ["station", "date", "max_tmpf", "min_tmpf", "precip", "name"]
weather_properties

['station', 'date', 'max_tmpf', 'min_tmpf', 'precip', 'name']

Execute extract fields function on raw weather dataframe for fields in weather properties list:

In [23]:
for i in weather_properties:
    extractFields(i)

View updated weather dataframe:

In [24]:
weather_raw

,type,properties,geometry,station,date,max_tmpf,min_tmpf,precip,name
0,Feature,"{'station': 'MN001', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-93.35405731...",MN001,2023-02-01,20.119978,-0.400011,NaN,Twin Lakes I-35 Mile Post 1
1,Feature,"{'station': 'MN002', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-94.1191, 44...",MN002,2023-02-01,13.460011,-9.039989,NaN,Silver Lake TH 7 Mile Post 1
2,Feature,"{'station': 'MN003', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-93.29242706...",MN003,2023-02-01,17.960010,-10.300011,NaN,Little Chicago I-35 Mile Post 70
3,Feature,"{'station': 'MN004', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-92.99275207...",MN004,2023-02-01,17.960010,-16.600012,NaN,Rush City I-35 Mile Post 157
4,Feature,"{'station': 'MN005', 'date': '2023-02-01', 'ma...","{'type': 'Point', 'coordinates': [-92.83856201...",MN005,2023-02-01,15.799989,-13.900011,NaN,Rutledge I-35 Mile Post 198
...,...,...,...,...,...,...,...,...,...
4279,Feature,"{'station': 'MN158', 'date': '2023-02-28', 'ma...","{'type': 'Point', 'coordinates': [-96.27693176...",MN158,2023-02-28,31.099989,22.819979,NaN,U.S.75 - Canby - MP 84.0 MN US MNDOT
4280,Feature,"{'station': 'MN159', 'date': '2023-02-28', 'ma...","{'type': 'Point', 'coordinates': [-94.81175994...",MN159,2023-02-28,31.999989,16.160011,NaN,U.S.12 - Atwater - MP 85.4 MN US MNDOT
4281,Feature,"{'station': 'MN160', 'date': '2023-02-28', 'ma...","{'type': 'Point', 'coordinates': [-96.04634094...",MN160,2023-02-28,31.999989,25.519978,NaN,U.S.14 - Florence - MP 21.2 MN US MNDOT
4282,Feature,"{'station': 'MN161', 'date': '2023-02-28', 'ma...","{'type': 'Point', 'coordinates': [-93.76606750...",MN161,2023-02-28,38.840023,17.419977,NaN,U.S.12 - Delano - MP 140.4 MN US MNDOT


Extract coordinates from geometry in dictionary:

In [25]:
weather_raw["x"] = weather_raw["geometry"].apply(lambda x: dict(x)["coordinates"][0])
weather_raw["y"] = weather_raw["geometry"].apply(lambda x: dict(x)["coordinates"][1])

Make copy of weather dataframe with desired columns and display updated table:

In [26]:
weather_df = weather_raw[["station", "date", "max_tmpf", "min_tmpf", "precip", "name", "x", "y"]].copy()

weather_df

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2023-02-01,20.119978,-0.400011,NaN,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2023-02-01,13.460011,-9.039989,NaN,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2023-02-01,17.960010,-10.300011,NaN,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2023-02-01,17.960010,-16.600012,NaN,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2023-02-01,15.799989,-13.900011,NaN,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
4279,MN158,2023-02-28,31.099989,22.819979,NaN,U.S.75 - Canby - MP 84.0 MN US MNDOT,-96.276932,44.674171
4280,MN159,2023-02-28,31.999989,16.160011,NaN,U.S.12 - Atwater - MP 85.4 MN US MNDOT,-94.811760,45.139050
4281,MN160,2023-02-28,31.999989,25.519978,NaN,U.S.14 - Florence - MP 21.2 MN US MNDOT,-96.046341,44.240311
4282,MN161,2023-02-28,38.840023,17.419977,NaN,U.S.12 - Delano - MP 140.4 MN US MNDOT,-93.766068,45.035450


Replace null precipitation values with 0:

In [27]:
weather_df["precip"].fillna(0, inplace=True)

Drop Latitude and Longitude rows that have null values:

In [28]:
weather_df = weather_df.dropna(subset=["x", "y"])

weather_df

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2023-02-01,20.119978,-0.400011,0.0,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2023-02-01,13.460011,-9.039989,0.0,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2023-02-01,17.960010,-10.300011,0.0,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2023-02-01,17.960010,-16.600012,0.0,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2023-02-01,15.799989,-13.900011,0.0,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
4279,MN158,2023-02-28,31.099989,22.819979,0.0,U.S.75 - Canby - MP 84.0 MN US MNDOT,-96.276932,44.674171
4280,MN159,2023-02-28,31.999989,16.160011,0.0,U.S.12 - Atwater - MP 85.4 MN US MNDOT,-94.811760,45.139050
4281,MN160,2023-02-28,31.999989,25.519978,0.0,U.S.14 - Florence - MP 21.2 MN US MNDOT,-96.046341,44.240311
4282,MN161,2023-02-28,38.840023,17.419977,0.0,U.S.12 - Delano - MP 140.4 MN US MNDOT,-93.766068,45.035450


Convert data types for station, name, and date columns:

In [29]:
# Convert to string
weather_df["station"] = weather_df["station"].astype(str)
weather_df["name"] = weather_df["name"].astype(str)

# Convert to 64-bit date/time integer data type
weather_df["date"] = weather_df["date"].astype('datetime64[ns]')

Remove rows in precipitation column that are less than 0:

In [30]:
weather_df = weather_df.loc[weather_df["precip"] >= 0]

weather_df

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2023-02-01,20.119978,-0.400011,0.0,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2023-02-01,13.460011,-9.039989,0.0,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2023-02-01,17.960010,-10.300011,0.0,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2023-02-01,17.960010,-16.600012,0.0,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2023-02-01,15.799989,-13.900011,0.0,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
4279,MN158,2023-02-28,31.099989,22.819979,0.0,U.S.75 - Canby - MP 84.0 MN US MNDOT,-96.276932,44.674171
4280,MN159,2023-02-28,31.999989,16.160011,0.0,U.S.12 - Atwater - MP 85.4 MN US MNDOT,-94.811760,45.139050
4281,MN160,2023-02-28,31.999989,25.519978,0.0,U.S.14 - Florence - MP 21.2 MN US MNDOT,-96.046341,44.240311
4282,MN161,2023-02-28,38.840023,17.419977,0.0,U.S.12 - Delano - MP 140.4 MN US MNDOT,-93.766068,45.035450


Remove outliers for max temperature:

In [31]:
max_tmpf_mn = weather_df["max_tmpf"].mean()
max_tmpf_std = weather_df["max_tmpf"].std()

In [32]:
weather_df = weather_df.loc[weather_df["max_tmpf"] < max_tmpf_mn + (max_tmpf_std * 3)]
weather_df = weather_df.loc[weather_df["max_tmpf"] > max_tmpf_mn - (max_tmpf_std * 3)]

weather_df

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2023-02-01,20.119978,-0.400011,0.0,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2023-02-01,13.460011,-9.039989,0.0,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2023-02-01,17.960010,-10.300011,0.0,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2023-02-01,17.960010,-16.600012,0.0,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2023-02-01,15.799989,-13.900011,0.0,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
4279,MN158,2023-02-28,31.099989,22.819979,0.0,U.S.75 - Canby - MP 84.0 MN US MNDOT,-96.276932,44.674171
4280,MN159,2023-02-28,31.999989,16.160011,0.0,U.S.12 - Atwater - MP 85.4 MN US MNDOT,-94.811760,45.139050
4281,MN160,2023-02-28,31.999989,25.519978,0.0,U.S.14 - Florence - MP 21.2 MN US MNDOT,-96.046341,44.240311
4282,MN161,2023-02-28,38.840023,17.419977,0.0,U.S.12 - Delano - MP 140.4 MN US MNDOT,-93.766068,45.035450


Remove outliers for min temperature:

In [33]:
min_tmpf_mn = weather_df["min_tmpf"].mean()
min_tmpf_std = weather_df["min_tmpf"].std()

In [34]:
weather_df = weather_df.loc[weather_df["min_tmpf"] < min_tmpf_mn + (min_tmpf_std * 3)]
weather_df = weather_df.loc[weather_df["min_tmpf"] > min_tmpf_mn - (min_tmpf_std * 3)]

weather_df

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2023-02-01,20.119978,-0.400011,0.0,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2023-02-01,13.460011,-9.039989,0.0,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2023-02-01,17.960010,-10.300011,0.0,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2023-02-01,17.960010,-16.600012,0.0,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2023-02-01,15.799989,-13.900011,0.0,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
4279,MN158,2023-02-28,31.099989,22.819979,0.0,U.S.75 - Canby - MP 84.0 MN US MNDOT,-96.276932,44.674171
4280,MN159,2023-02-28,31.999989,16.160011,0.0,U.S.12 - Atwater - MP 85.4 MN US MNDOT,-94.811760,45.139050
4281,MN160,2023-02-28,31.999989,25.519978,0.0,U.S.14 - Florence - MP 21.2 MN US MNDOT,-96.046341,44.240311
4282,MN161,2023-02-28,38.840023,17.419977,0.0,U.S.12 - Delano - MP 140.4 MN US MNDOT,-93.766068,45.035450


Remove outliers for precipitation:

In [35]:
precipitation_mn = weather_df["precip"].mean()
precipitation_std = weather_df["precip"].std()

In [36]:
weather_df = weather_df.loc[weather_df["precip"] < precipitation_mn + (precipitation_std * 3)]

weather_df

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2023-02-01,20.119978,-0.400011,0.0,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2023-02-01,13.460011,-9.039989,0.0,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2023-02-01,17.960010,-10.300011,0.0,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2023-02-01,17.960010,-16.600012,0.0,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2023-02-01,15.799989,-13.900011,0.0,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
4279,MN158,2023-02-28,31.099989,22.819979,0.0,U.S.75 - Canby - MP 84.0 MN US MNDOT,-96.276932,44.674171
4280,MN159,2023-02-28,31.999989,16.160011,0.0,U.S.12 - Atwater - MP 85.4 MN US MNDOT,-94.811760,45.139050
4281,MN160,2023-02-28,31.999989,25.519978,0.0,U.S.14 - Florence - MP 21.2 MN US MNDOT,-96.046341,44.240311
4282,MN161,2023-02-28,38.840023,17.419977,0.0,U.S.12 - Delano - MP 140.4 MN US MNDOT,-93.766068,45.035450


Include only rows that fall within bounding box of Minnesota:

In [37]:
weather_df = weather_df.loc[weather_df["x"] > -97.5]
weather_df = weather_df.loc[weather_df["x"] < -89.0]
weather_df = weather_df.loc[weather_df["y"] > 43.0]
weather_df = weather_df.loc[weather_df["y"] < 49.5]

weather_df

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2023-02-01,20.119978,-0.400011,0.0,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2023-02-01,13.460011,-9.039989,0.0,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2023-02-01,17.960010,-10.300011,0.0,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2023-02-01,17.960010,-16.600012,0.0,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2023-02-01,15.799989,-13.900011,0.0,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
4279,MN158,2023-02-28,31.099989,22.819979,0.0,U.S.75 - Canby - MP 84.0 MN US MNDOT,-96.276932,44.674171
4280,MN159,2023-02-28,31.999989,16.160011,0.0,U.S.12 - Atwater - MP 85.4 MN US MNDOT,-94.811760,45.139050
4281,MN160,2023-02-28,31.999989,25.519978,0.0,U.S.14 - Florence - MP 21.2 MN US MNDOT,-96.046341,44.240311
4282,MN161,2023-02-28,38.840023,17.419977,0.0,U.S.12 - Delano - MP 140.4 MN US MNDOT,-93.766068,45.035450


Convert from DF to Feature Class:

In [38]:
# Convert from DF to SEDF
weather_sedf = arcgis.GeoAccessor.from_xy(weather_df, "x", "y")

In [39]:
# Convert from SEDF to Feature Class
weather_sedf.spatial.to_featureclass(location=os.path.join(local_gdb, "weather_observations"))

'C:\\Users\\cason\\Documents\\GIS5572\\Lab2\\Lab2.gdb\\weather_observations'

# Rasters

Define function that performs raster QA/QC. Checking for null values, cell size, outliers, spatial reference, and bounding box:

In [40]:
def check_raster(raster_path, bounding_box, expected_width, expected_height, categorical=False, expected_crs=None):
    # Check for null values
    null_count = arcpy.GetRasterProperties_management(raster_path, "ANYNODATA").getOutput(0)
    if int(null_count) > 0:
        print("Raster contains null values")
    
    # Check cell size
    desc = arcpy.Describe(raster_path)
    cell_width = desc.meanCellWidth
    cell_height = desc.meanCellHeight
    if cell_width != expected_width or cell_height != expected_height:
        print("Raster cell size is not correct")
    
    # Check for outliers
    if not categorical:
        mean = arcpy.GetRasterProperties_management(raster_path, "MEAN").getOutput(0)
        std_dev = arcpy.GetRasterProperties_management(raster_path, "STD").getOutput(0)
        min_val = arcpy.GetRasterProperties_management(raster_path, "MINIMUM").getOutput(0)
        max_val = arcpy.GetRasterProperties_management(raster_path, "MAXIMUM").getOutput(0)
        if (float(max_val) > float(mean) + 3 * float(std_dev)) or (float(min_val) < float(mean) - 3 * float(std_dev)):
            print("Raster contains outliers")
    
    # Check CRS
    if expected_crs is not None:
        spatial_ref = desc.spatialReference
        if spatial_ref.name != expected_crs:
            print("Raster CRS is not correct")
    
    # Check if raster is within bounding box
    extent = desc.extent
    if not (extent.XMin >= bounding_box[0] and extent.XMax <= bounding_box[1] and extent.YMin >= bounding_box[2] and extent.YMax <= bounding_box[3]):
        print("Raster is not within bounding box")

Check DEM for null values and outliers, as well as checking if it fits MN bounding box and has correct spatial reference:

In [41]:
check_raster(
    raster_path=dem, 
    bounding_box=(-97.5, 43.0, -89.00, 49.5), 
    expected_width=30, 
    expected_height=30, 
    categorical=False, 
    expected_crs="NAD_1983_UTM_Zone_15N"
)

Raster contains null values
Raster contains outliers
Raster is not within bounding box


Check NLCD for null values and whether it fits MN bounding box and has correct spatial reference:

In [42]:
check_raster(
    raster_path=landcover_path, 
    bounding_box=(-97.5, 43.0, -89.00, 49.5), 
    expected_width=30, 
    expected_height=30, 
    categorical=True, 
    expected_crs="NAD_1983_UTM_Zone_15N"
)

Raster contains null values
Raster is not within bounding box


Clip NLCD to MN bounding box:

In [43]:
arcpy.management.Clip(
    landcover_path, 
    "132660 4774410 791819 5491608", 
    os.path.join(
        local_gdb, 
        "landcover_final"
    )
)

<Result 'C:\\Users\\cason\\Documents\\GIS5572\\Lab2\\Lab2.gdb\\landcover_final'>

Clip DEM to MN bounding box:

In [44]:
arcpy.management.Clip(
    dem, 
    "132660 4774410 791819 5491608", 
    os.path.join(
        local_gdb, 
        "dem_final"
    )
)

<Result 'C:\\Users\\cason\\Documents\\GIS5572\\Lab2\\Lab2.gdb\\dem_final'>

# Export to PostgreSQL database from Local GDB

In [47]:
# Specify path for PostgreSQL database
sde = r"C:\Users\cason\Documents\GIS5572\Lab2\34.135.163.144.sde"

# Export vector data to PostgreSQL database
arcpy.conversion.FeatureClassToGeodatabase(
    f'{os.path.join(local_gdb, "weather_observations")};{os.path.join(local_gdb, "bare_soil_observations")}',
    sde
)

# Export raster data to PostgreSQL database
arcpy.conversion.RasterToGeodatabase(
    f'{os.path.join(local_gdb, "landcover_final")};{os.path.join(local_gdb, "dem_final")}',
    sde
)

<Result ''>